In [1]:
import marvin
marvin.config.download = True
marvin.config.release = 'DR17'

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.stats as sci

from marvin.tools.vacs import VACs
from astropy.io import fits
import numpy as np
from marvin.tools import Maps
from marvin.tools import Cube

from marvin import config #Needed to fix "Cube not subscriptable" when running in Sciserver
config.forceDbOff() #Needed to fix "Cube not subscriptable" when running in Sciserver

print('Marvin and Matplotlib imported. Warnings about missing paths/files are typical, so no need to be concerned')

/home/idies/miniconda3/envs/py38/lib/python3.8/site-packages/marvin/core/exceptions.py:50: UserWarning: cannot initiate Sentry error reporting: [Errno 6] No such device or address.
  warnings.warn('cannot initiate Sentry error reporting: {0}.'.format(str(ee)),
[INFO]: No release version set. Setting default to DR17
[WARNING]: path /home/idies/workspace/sdss_sas/mangawork/manga/spectro/redux/v3_1_1/drpall-v3_1_1.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/idies/workspace/sdss_sas/mangawork/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits cannot be found. Setting dapall to None. (MarvinUserWarning)


Marvin and Matplotlib imported. Warnings about missing paths/files are typical, so no need to be concerned


In [2]:
selection = 'full_initial_sample.fits'
hdu = fits.open(selection)
data = hdu[1].data

mangaplifu = []
plates = []
ifudsgn=[]

for galaxy in range(len(data)):
    if data[galaxy]['GZ_SPIRAL_VOTES']/data[galaxy]['GZ_TOTAL_CLASSIFICATIONS']>0.2 and data[galaxy]['IFUDESIGNSIZE_1']>0 and data[galaxy]['nsa_elpetro_ba']>0.5:
        mangaplifu.append(str(data[galaxy]['plate_1'])+'-'+str(data[galaxy]['ifudsgn_1']))
        plates.append(data[galaxy]['plate_1'])
        ifudsgn.append(data[galaxy]['ifudsgn_1'])
        
hdu.close()
#selecting galaxies with spiral arms using GZ3D metadata

In [3]:
#a function to get appropriate values for arm vs interarm spaxels
def arm_selector (mangaid, plate, ifudsgn):
    pi3D = 'https://data.sdss.org/sas/dr17/manga/spectro/pipe3d/v3_1_1/3.1.1/'+str(plate)+'/manga-'+str(plate)+'-'+str(ifudsgn)+'.Pipe3D.cube.fits.gz'
    #Getting the FITS files for Pipe3D
    hdu1 = fits.open(pi3D)
    mass = hdu1[1].data[19]
    
    #Getting the GZ3D files
    maps = marvin.tools.Maps(mangaid,mode='remote')
    data = maps.vacs.gz3d
    
    #selecting arm vs interarm spaxels
    galaxy = data.spiral_mask_spaxel
    arm = np.where(galaxy > 5,1,np.nan)
    interarm = np.where(galaxy < 3,1,np.nan)
    ma = mass*arm
    mi= mass*interarm
    
    #getting binned averages
    radius = maps.spx_ellcoo_r_re
    mask=np.where(np.isnan(ma),0,1)
    mask2=np.where(np.isnan(mi),0,1)
    mass_ma,radius_ma,binnumber = sci.binned_statistic(radius.value[mask!=0],ma[mask!=0],bins=20,range=[0.5,2.5])
    mass_mi,radius_mi,binnnumber = sci.binned_statistic(radius.value[mask2!=0],mi[mask2!=0],bins=20,range=[0.5,2.5])
    
    hdu1.close()
    
    return mass_ma,radius_ma,mass_mi,radius_mi

In [5]:
mass_ma_list=[]
mass_mi_list=[]
rad_ma_list=[]
rad_mi_list=[]

file = open('arm_vs_interarm.txt','a')

for i in range(237,248):
    mass_ma,radius_ma,mass_mi,radius_mi= arm_selector(mangaplifu[i],plates[i],ifudsgn[i])
    f=((10**np.log10(mass_ma))-(10**np.log10(mass_mi)))/(10**np.log10(mass_mi))
    file.write(str(mangaplifu[i]))
    file.write('\n')
    file.write('Arm Mass=')
    file.write(str(mass_ma))
    file.write('\n')
    file.write('Radius=')
    file.write(str(radius_ma))
    file.write('\n')
    file.write('Interarm Mass=')
    file.write(str(mass_mi))
    file.write('\n')
    file.write('Excess in Mass=')
    file.write(str(f))
    file.write('\n')
    file.write('\n')
file.close()
#Getting the average mass and radius values and the excess in mass values

Progress: 100% 1/1 [00:10<00:00, 10.02s/files, n_files=1, n_streams=1]


SDSS_ACCESS> Done!


[INFO]: downloading file for VAC 'gz3d'
Progress: 100% 1/1 [00:05<00:00,  5.01s/files, n_files=1, n_streams=1]


SDSS_ACCESS> Done!


Progress: 100% 1/1 [00:10<00:00, 10.02s/files, n_files=1, n_streams=1]


...

SDSS_ACCESS> Done!


Progress: 100% 1/1 [00:05<00:00,  5.01s/files, n_files=1, n_streams=1]


SDSS_ACCESS> Done!


[INFO]: downloading file for VAC 'gz3d'
Progress: 100% 1/1 [00:05<00:00,  5.01s/files, n_files=1, n_streams=1]


SDSS_ACCESS> Done!
